![Add a relevant banner image here](path_to_image)

# Project Title

## Overview

This project seeks to build machine learning models to achieve two things:

1. Predict the highest ranking a song will achieve on the Billboard Hot 100 list.
2. Predict the largest week over week increase in a given song's ranking on the Hot 100 list.

Key Insights:

- A song's genre has a minimal impact on its ranking on the Billboard Hot 100 list.
- Song characteristics such as tempo, danceability, etc. appear to be the strongest drivers of their performance on the Hot 100 list.
- Analyzing music can quickly lead to an overwhelming number of features, so thoughtful and careful data selection is key.

## Business Understanding

The customer of this project is FutureProduct Advisors, a consultancy that helps their customers develop innovative and new consumer products. FutureProduct’s customers are increasingly seeking help from their consultants in go-to-market activities. 

FutureProduct’s consultants can support these go-to-market activities, but the business does not have all the infrastructure needed to support it. Their biggest ask is for a tool to help them find interesting, up-and-coming music to accompany social posts and online ads for go-to-market promotions. 

**Stakeholders**

- FutureProduct Managing Director: oversees their consulting practice and is sponsoring this project.
- FutureProduct Senior Consultants: the actual users of the prospective tool. A small subset of the consultants will pilot the prototype tool.
- My consulting leadership: sponsors of this effort; will provide oversight and technical input of the project as needed.

**Primary Goals**

1.	Build a data tool that can evaluate any song in the Billboard Hot 100 list and make predictions about:
    -	The song’s position on the Hot 100 list 4 weeks in the future
    -	The song’s highest position on the list in the next 6 months
2.	Create a rubric that lists the 3 most important factors for songs’ placement on the Hot 100 list for each hear from 2000 to 2021.


## Data Understanding

Billboard Hot 100 weekly charts (Kaggle): https://www.kaggle.com/datasets/thedevastator/billboard-hot-100-audio-features

I’ve chosen this dataset because it has a direct measurement of song popularity (the Hot 100 list) and because its long history gives significant context to a song’s positioning in a given week.
The features list gives a wide range of song attributes to explore and enables me to determine what features most significantly contribute to a song’s popularity and how that changes over time.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import ast
from collections import Counter

import xgboost as xgb

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay, mean_squared_error, r2_score, pairwise_distances

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

np.random.seed(42)

In [3]:
df_hotlist_all = pd.read_csv('Data/Hot Stuff.csv')
df_features_all = pd.read_csv('Data/Hot 100 Audio Features.csv')

In [ ]:
# exploring hotlist df
df_hotlist_all.info()

In [ ]:
# exploring features df
df_features_all.info()

#### Exploratory Data Analysis

In [ ]:
liveness_dist = df_cleaned_genre['liveness'].value_counts()
df_liveness_dist = pd.DataFrame(liveness_dist)
df_liveness_dist = df_liveness_dist.reset_index()

plt.bar(df_liveness_dist['liveness'], df_liveness_dist['count'], color='orange')
plt.xlabel('Liveness Rating')
plt.ylabel('Number of Songs')
plt.title('Distribution of Liveness Ratings')
plt.show()

In [ ]:
danceability_dist = df_cleaned_genre['danceability'].value_counts()
df_danceability_dist = pd.DataFrame(danceability_dist)
df_danceability_dist = df_danceability_dist.reset_index()

plt.bar(df_danceability_dist['danceability'], df_danceability_dist['count'], color='skyblue')
plt.xlabel('Danceability Rating')
plt.ylabel('Number of Songs')
plt.title('Distribution of Danceability Ratings')
plt.show()

In [ ]:
acousticness_dist = df_cleaned_genre['acousticness'].value_counts()
df_acousticness_dist = pd.DataFrame(acousticness_dist)
df_acousticness_dist = df_acousticness_dist.reset_index()

plt.bar(df_acousticness_dist['acousticness'], df_acousticness_dist['count'], color='orange')
plt.xlabel('Acousticness Rating')
plt.ylabel('Number of Songs')
plt.title('Distribution of Acousticness Ratings')
plt.show()

In [ ]:
peak_pos_dist = df_cleaned_genre['Max_Peak_Position'].value_counts()
df_peak_pos_dist = pd.DataFrame(peak_pos_dist)
df_peak_pos_dist = df_peak_pos_dist.reset_index()
print(f"Mean Highest Ranking: {df_cleaned_genre['Max_Peak_Position'].mean():.0f}")
print(f"Standard Deviation of Highest Ranking: {df_cleaned_genre['Max_Peak_Position'].std():.1f}")

In [ ]:
plt.bar(df_peak_pos_dist['Max_Peak_Position'], df_peak_pos_dist['count'], color='skyblue')
plt.xlabel('Highest Ranking')
plt.ylabel('Number of Songs')
plt.title('Frequency of Peak Rankings')

In [ ]:
max_rank_change = df_cleaned_genre['Max_Rank_Change'].value_counts()
df_max_rank_change = pd.DataFrame(max_rank_change)
df_max_rank_change = df_max_rank_change.reset_index()
df_max_rank_change = df_max_rank_change[df_max_rank_change['Max_Rank_Change'] > 0]
print(f"Mean Largest Week over Week Rank Change: {df_cleaned_genre['Max_Rank_Change'].mean():.0f}")
print(f"Standard Deviation of Largest Week over Week Rank Change: {df_cleaned_genre['Max_Rank_Change'].std():.1f}")

In [ ]:
plt.bar(df_max_rank_change['Max_Rank_Change'], df_max_rank_change['count'], color='orange')
plt.xlabel('Largest Week over Week Rank Change')
plt.ylabel('Number of Songs')
plt.title('Frequency of Largest Week over Week Rank Change')


## Data Preparation

### Initial Data Selection

In [4]:
# removing hotlist df attributes that will not be used in cleaning or analysis
df_hotlist_all = df_hotlist_all.drop(['index', 'url', 'Song', 'Performer', 'Instance'], axis=1)
# converting WeekID to datetime
df_hotlist_all['WeekID'] = pd.to_datetime(df_hotlist_all['WeekID'], errors='coerce')
df_hotlist_all = df_hotlist_all.sort_values(by='WeekID')

# creating a new hotlist df with only complete year data from 2000 - 2020, the time period being studied
df_hotlist_2000s = df_hotlist_all.loc[(df_hotlist_all['WeekID'] > '1999-12-31') & (df_hotlist_all['WeekID'] < '2021-01-01')]

# adding a column to calculate the week over week change in rank
def diff(a, b):
    return a - b

df_hotlist_2000s['Rank_Change'] = df_hotlist_2000s.apply(lambda x: diff(x['Week Position'], x['Previous Week Position']), axis=1)
# replacing NaNs with 0
df_hotlist_2000s['Rank_Change'] = df_hotlist_2000s['Rank_Change'].fillna(0)

# removing features df attributes that will not be used in cleaning or analysis
df_features_all = df_features_all.drop(['index', 'Performer', 'Song', 'spotify_track_album', 
                                        'spotify_track_id', 'spotify_track_preview_url',  
                                        'spotify_track_explicit', 'spotify_track_popularity'], axis=1)


C:\Users\marha\AppData\Local\Temp\ipykernel_18008\3177740419.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hotlist_2000s['Rank_Change'] = df_hotlist_2000s.apply(lambda x: diff(x['Week Position'], x['Previous Week Position']), axis=1)
C:\Users\marha\AppData\Local\Temp\ipykernel_18008\3177740419.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hotlist_2000s['Rank_Change'] = df_hotlist_2000s['Rank_Change'].fillna(0)


In [5]:
# combining the hotlist and features into one dataframe

df_hotlist_and_features_2000s = pd.merge(df_hotlist_2000s, df_features_all, on='SongID', how='left')
df_hotlist_and_features_2000s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112098 entries, 0 to 112097
Data columns (total 21 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   WeekID                     112098 non-null  datetime64[ns]
 1   Week Position              112098 non-null  int64         
 2   SongID                     112098 non-null  object        
 3   Previous Week Position     101571 non-null  float64       
 4   Peak Position              112098 non-null  int64         
 5   Weeks on Chart             112098 non-null  int64         
 6   Rank_Change                112098 non-null  float64       
 7   spotify_genre              108091 non-null  object        
 8   spotify_track_duration_ms  104590 non-null  float64       
 9   danceability               104287 non-null  float64       
 10  energy                     104287 non-null  float64       
 11  key                        104287 non-null  float64 

### Feature Engineering

In [6]:
"""
The dataset has genre in a single column and the entry for each song has a variety of genres listed in that single column. 
This does not allow me to explore genre in a systematic way.
I'll need to break genre out so that each genre has its own column with a 1 or 0 to indicate whether each song is tagged with that genre 
(ending with a one-hot encoded structure).
"""

# generating a df with unique genre names
unique_genres = list(set(
    genre 
    for genre_string in df_hotlist_and_features_2000s['spotify_genre'] 
    if pd.notna(genre_string)
    for genre in ast.literal_eval(genre_string)
))

df_unique_genres = pd.DataFrame(unique_genres, columns=['genre'])

# adding counts of each unique genre name
# Extract all genres (with duplicates) and count them
all_genres_list = []
for genre_string in df_hotlist_and_features_2000s['spotify_genre']:
    if pd.notna(genre_string):
        genre_list = ast.literal_eval(genre_string)
        all_genres_list.extend(genre_list)

# Count occurrences
genre_counts = Counter(all_genres_list)

# Map counts to genres dataframe
df_unique_genres['count'] = df_unique_genres['genre'].map(genre_counts)
df_unique_genres = df_unique_genres.sort_values('count', ascending=False)

In [ ]:
# writing to csv for easier review of the data
df_unique_genres.to_csv('genre_counts.csv', index=False)

After reviewing the full set of genre counts, I'm only including genres that appear in 100 or more songs (i.e. at least 0.1% of songs).

In [7]:
# loading list of genres with 100 or more instances in df_cleaned
df_genres_100_up = pd.read_csv('genre_counts_100+inst.csv')

# converting df to list
final_genres_list = df_genres_100_up['genre'].tolist()

# manually one-hot encoding each genre

# creating a list of genres and counts
genre_data = []

for genre_string in df_hotlist_and_features_2000s['spotify_genre'] :
    if pd.notna(genre_string):
        genre_list = ast.literal_eval(genre_string)
        row_dict = {genre: (1 if genre in genre_list else 0) for genre in final_genres_list} # dict with 1 if genre exists in list, 0 if not
    else:
         row_dict = {genre: 0 for genre in final_genres_list} # 0 of genre does not exist in list
    genre_data.append(row_dict)

# creating a df with the list of dicts
genre_df = pd.DataFrame(genre_data)

# concatenating genre data into df_clean
df_hotlist_and_features_2000s = pd.concat([df_hotlist_and_features_2000s, genre_df], axis=1)


In [8]:
# removing spotify_genre and spot-checking resulting df 
pd.set_option('display.max_columns', None)
df_hotlist_and_features_2000s = df_hotlist_and_features_2000s.drop(['spotify_genre'], axis=1)
df_hotlist_and_features_2000s.head()

,WeekID,Week Position,SongID,Previous Week Position,Peak Position,Weeks on Chart,Rank_Change,spotify_track_duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,pop,dance pop,pop rap,rap,contemporary country,country,country road,post-teen pop,hip hop,r&b,urban contemporary,trap,southern hip hop,pop rock,hip pop,modern country rock,neo mellow,post-grunge,gangster rap,atl hip hop,alternative metal,neo soul,dirty south rap,country dawn,modern rock,nu metal,canadian pop,rock,melodic rap,deep pop r&b,edm,new jack swing,permanent wave,miami hip hop,country pop,oklahoma country,latin,tropical house,electropop,uk pop,east coast hip hop,alternative rock,viral pop,quiet storm,chicago rap,redneck,pop punk,crunk,country rock,toronto rap,canadian hip hop,boy band,hardcore hip hop,queens hip hop,talent show,emo,europop,acoustic pop,alternative r&b,conscious hip hop,australian pop,detroit hip hop,rap rock,latin pop,barbadian pop,g funk,girl group,canadian rock,tropical,piano rock,indie pop,electro house,indie poptimism,rap metal,west coast rap,new orleans rap,metropopolis,candy pop,lilith,australian country,philly rap,funk metal,reggaeton,dfw rap,canadian contemporary r&b,soul,mexican pop,adult standards,nc hip hop,british soul,trap queen,hollywood,arkansas country,atl trap,underground hip hop,texas country,uk dance,house,new wave pop,brostep,dancehall,progressive house,funk,singer-songwriter,latin hip hop,idol,garage rock,mellow gold,baroque pop,big room,art pop,reggae fusion,cali rap,bronx hip hop,folk-pop,country rap,stomp and holler,neon pop punk,emo rap,punk,indie rock,funk rock,memphis hip hop,modern alternative rock,lgbtq+ hip hop,progressive electro house,alternative hip hop,blues rock,colombian pop,eurodance,classic rock,baton rouge rap,australian dance,folk,pop emo,soft rock,motown,pixie,canadian country,wrestling,glee club,complextro,vapor trap,etherpop,pittsburgh rap,escape room,indietronica,comic,german techno,new jersey rap,trap latino,houston rap,social media pop,puerto rican pop,deep southern trap,heartland rock,alternative dance,bubblegum dance,alberta country,outlaw country,country gospel,florida rap,hard rock,canadian metal,christian rock,soca,indiecoustica,harlem hip hop,new rave,electronic trap,christian music,grunge,show tunes,viral trap,la indie,swedish pop,swedish electropop,reggaeton flow,dance-punk,celtic rock,socal pop punk,lounge,chicano rap,stomp pop,ccm,vocal jazz,glam metal,worship,irish rock,electropowerpop,electro,indie pop rap,canadian contemporary country,bounce,christian alternative rock,south african rock,deep talent show,disco,hyphy,disco house,canadian latin,australian hip hop,nyc rap,brill building pop,k-pop,nz pop,minnesota hip hop,modern blues rock,album rock,modern folk rock,uk americana,old school hip hop,punk blues,dmv rap,industrial metal,skate punk,swedish synthpop,moombahton
0,2000-01-01,69,Deck The HallsSHeDAISY,97.0,69,2,-28.0,229773.0,0.575,0.837,1.0,-7.141,0.0,0.0406,0.0195,0.000009,0.144,0.444,118.827,4.0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2000-01-01,83,Guerrilla RadioRage Against The Machine,87.0,69,10,-4.0,206200.0,0.599,0.957,11.0,-5.764,1.0,0.1880,0.0129,0.000071,0.155,0.489,103.680,4.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [9]:
# creating new df with most 5 most popular genres by week

# Get all genre column names
genre_start_idx = df_hotlist_and_features_2000s.columns.get_loc('pop')
genre_cols = df_hotlist_and_features_2000s.columns[genre_start_idx:].tolist()

# Group by WeekID and sum the genre columns to get counts
genre_counts = df_hotlist_and_features_2000s.groupby('WeekID')[genre_cols].sum()

# For each week, find the top 5 genres
top_genres = []
for week_id in genre_counts.index:
    # Get the genre counts for this week and sort them
    week_genres = genre_counts.loc[week_id].sort_values(ascending=False)
    
    # Get the top 3 genre names
    top_5 = week_genres.head(5).index.tolist()
    
    # Pad with None if there are fewer than 5 genres
    while len(top_5) < 5:
        top_5.append(None)
    
    top_genres.append({
        'WeekID': week_id,
        'Most_Popular_Genre': top_5[0],
        '2nd_Most_Popular_Genre': top_5[1],
        '3rd_Most_Popular_Genre': top_5[2],
        '4th_Most_Popular_Genre': top_5[3],
        '5th_Most_Popular_Genre': top_5[4]
    })

# Create the new dataframe
df_top_genres = pd.DataFrame(top_genres)


In [10]:
# adding a column to the main df indicating whether each song is in a genre that's in the top 5 genres for a given week

def is_in_top5_genres(row, df_top_genres):
    week = row['WeekID']

    top_genres = df_top_genres[df_top_genres['WeekID'] == week]

    if len(top_genres) == 0:
        return 0
    
    top_5 = [
        top_genres.iloc[0]['Most_Popular_Genre'],
        top_genres.iloc[0]['2nd_Most_Popular_Genre'],
        top_genres.iloc[0]['3rd_Most_Popular_Genre'],
        top_genres.iloc[0]['4th_Most_Popular_Genre'],
        top_genres.iloc[0]['5th_Most_Popular_Genre'],
        ]

    for genre in top_5:
        if genre in row.index and row[genre] == 1:
            return 1
        
        else:
            return 0
        
df_hotlist_and_features_2000s['in_top5_genres'] = df_hotlist_and_features_2000s.apply(
    lambda row: is_in_top5_genres(row, df_top_genres), axis=1
)

In [11]:
# new df with mean score of appearance in top 5 weekly genres
df_mean_genre_match = df_hotlist_and_features_2000s.groupby('SongID', as_index=False)['in_top5_genres'].mean()
df_mean_genre_match.rename(columns={'in_top5_genres': 'In_Top5genres_Mean'}, inplace=True)
df_mean_genre_match.set_index('SongID', inplace=True)

# new df with the max weekly rank change for each song 
df_max_rank_change = df_hotlist_and_features_2000s.groupby('SongID', as_index=False)['Rank_Change'].max()
df_max_rank_change.rename(columns={'Rank_Change': 'Max_Rank_Change'}, inplace=True)
df_max_rank_change.set_index('SongID', inplace=True)

# new df with the max peak rank for each song 
df_max_peak_pos = df_hotlist_and_features_2000s.groupby('SongID', as_index=False)['Peak Position'].max()
df_max_peak_pos.rename(columns={'Peak Position': 'Max_Peak_Position'}, inplace=True)
df_max_peak_pos.set_index('SongID', inplace=True)

# ensuring these new dfs have no null values
df_max_rank_change['Max_Rank_Change'].isna().sum(), df_max_peak_pos['Max_Peak_Position'].isna().sum(), df_mean_genre_match['In_Top5genres_Mean'].isna().sum()

(0, 0, 0)

In [12]:
# adding max peak position to main df
df_2000s_data = df_hotlist_and_features_2000s.join(df_max_peak_pos, on='SongID')

# adding max rank change to main df
df_2000s_data = df_2000s_data.join(df_max_rank_change, on='SongID')

# adding in top5genres mean to main df
df_2000s_data = df_2000s_data.join(df_mean_genre_match, on='SongID')

df_2000s_data.head()

,WeekID,Week Position,SongID,Previous Week Position,Peak Position,Weeks on Chart,Rank_Change,spotify_track_duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,pop,dance pop,pop rap,rap,contemporary country,country,country road,post-teen pop,hip hop,r&b,urban contemporary,trap,southern hip hop,pop rock,hip pop,modern country rock,neo mellow,post-grunge,gangster rap,atl hip hop,alternative metal,neo soul,dirty south rap,country dawn,modern rock,nu metal,canadian pop,rock,melodic rap,deep pop r&b,edm,new jack swing,permanent wave,miami hip hop,country pop,oklahoma country,latin,tropical house,electropop,uk pop,east coast hip hop,alternative rock,viral pop,quiet storm,chicago rap,redneck,pop punk,crunk,country rock,toronto rap,canadian hip hop,boy band,hardcore hip hop,queens hip hop,talent show,emo,europop,acoustic pop,alternative r&b,conscious hip hop,australian pop,detroit hip hop,rap rock,latin pop,barbadian pop,g funk,girl group,canadian rock,tropical,piano rock,indie pop,electro house,indie poptimism,rap metal,west coast rap,new orleans rap,metropopolis,candy pop,lilith,australian country,philly rap,funk metal,reggaeton,dfw rap,canadian contemporary r&b,soul,mexican pop,adult standards,nc hip hop,british soul,trap queen,hollywood,arkansas country,atl trap,underground hip hop,texas country,uk dance,house,new wave pop,brostep,dancehall,progressive house,funk,singer-songwriter,latin hip hop,idol,garage rock,mellow gold,baroque pop,big room,art pop,reggae fusion,cali rap,bronx hip hop,folk-pop,country rap,stomp and holler,neon pop punk,emo rap,punk,indie rock,funk rock,memphis hip hop,modern alternative rock,lgbtq+ hip hop,progressive electro house,alternative hip hop,blues rock,colombian pop,eurodance,classic rock,baton rouge rap,australian dance,folk,pop emo,soft rock,motown,pixie,canadian country,wrestling,glee club,complextro,vapor trap,etherpop,pittsburgh rap,escape room,indietronica,comic,german techno,new jersey rap,trap latino,houston rap,social media pop,puerto rican pop,deep southern trap,heartland rock,alternative dance,bubblegum dance,alberta country,outlaw country,country gospel,florida rap,hard rock,canadian metal,christian rock,soca,indiecoustica,harlem hip hop,new rave,electronic trap,christian music,grunge,show tunes,viral trap,la indie,swedish pop,swedish electropop,reggaeton flow,dance-punk,celtic rock,socal pop punk,lounge,chicano rap,stomp pop,ccm,vocal jazz,glam metal,worship,irish rock,electropowerpop,electro,indie pop rap,canadian contemporary country,bounce,christian alternative rock,south african rock,deep talent show,disco,hyphy,disco house,canadian latin,australian hip hop,nyc rap,brill building pop,k-pop,nz pop,minnesota hip hop,modern blues rock,album rock,modern folk rock,uk americana,old school hip hop,punk blues,dmv rap,industrial metal,skate punk,swedish synthpop,moombahton,in_top5_genres,Max_Peak_Position,Max_Rank_Change,In_Top5genres_Mean
0,2000-01-01,69,Deck The HallsSHeDAISY,97.0,69,2,-28.0,229773.0,0.575,0.837,1.0,-7.141,0.0,0.0406,0.0195,0.000009,0.144,0.444,118.827,4.0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,69,-8.0,0.0
1,2000-01-01,83,Guerrilla RadioRage Against The Machine,87.0,69,10,-4.0,206200.0,0.599,0.957,11.0,-5.764,1.0,0.1880,0.0129,0.000071,0.155,0.489,103.680,4.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [16]:
# creating a clean df with genre information
df_clean_withgenre = df_2000s_data.drop(['WeekID', 'Week Position', 'Previous Week Position', 'Peak Position',
                                         'Weeks on Chart', 'Rank_Change', 'in_top5_genres'], axis=1)

# counting duplicates and all rows
df_clean_withgenre.duplicated().sum(), len(df_clean_withgenre)

(103420, 112098)

In [17]:
# removing duplicate rows
df_clean_withgenre = df_clean_withgenre.drop_duplicates()

# checking duplicate rows and rows with missing values
print(f"Total rows: {len(df_clean_withgenre)}")
print(f"Duplicate rows: {df_clean_withgenre.duplicated().sum()}")
print(f"Rows with missing values: {df_clean_withgenre.isna().any(axis=1).sum()}")

Total rows: 8678
Duplicate rows: 0
Rows with missing values: 820


In [18]:
# dropping rows with missing values (unfortunately there is no reliable way to infer or estimate song characteristics)
df_clean_withgenre = df_clean_withgenre.dropna()

# re-checking duplicate rows and rows with missing values
print(f"Total rows: {len(df_clean_withgenre)}")
print(f"Duplicate rows: {df_clean_withgenre.duplicated().sum()}")
print(f"Rows with missing values: {df_clean_withgenre.isna().any(axis=1).sum()}")

Total rows: 7858
Duplicate rows: 0
Rows with missing values: 0


In [19]:
# creating a clean df with no genre
nogenre_cols = ['spotify_track_duration_ms', 'danceability', 'energy', 'key', 'loudness',
                'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 
                'time_signature', 'Max_Peak_Position', 'Max_Rank_Change', 'In_Top5genres_Mean']
df_clean_nogenre = df_clean_withgenre[nogenre_cols].copy()

I now have two datasets: one containing genre and one without. This will allow me to model this data with and without genre.

In [20]:
df_clean_withgenre = df_clean_withgenre.drop(['SongID'], axis=1)
df_clean_withgenre.describe()

,spotify_track_duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,pop,dance pop,pop rap,rap,contemporary country,country,country road,post-teen pop,hip hop,r&b,urban contemporary,trap,southern hip hop,pop rock,hip pop,modern country rock,neo mellow,post-grunge,gangster rap,atl hip hop,alternative metal,neo soul,dirty south rap,country dawn,modern rock,nu metal,canadian pop,rock,melodic rap,deep pop r&b,edm,new jack swing,permanent wave,miami hip hop,country pop,oklahoma country,latin,tropical house,electropop,uk pop,east coast hip hop,alternative rock,viral pop,quiet storm,chicago rap,redneck,pop punk,crunk,country rock,toronto rap,canadian hip hop,boy band,hardcore hip hop,queens hip hop,talent show,emo,europop,acoustic pop,alternative r&b,conscious hip hop,australian pop,detroit hip hop,rap rock,latin pop,barbadian pop,g funk,girl group,canadian rock,tropical,piano rock,indie pop,electro house,indie poptimism,rap metal,west coast rap,new orleans rap,metropopolis,candy pop,lilith,australian country,philly rap,funk metal,reggaeton,dfw rap,canadian contemporary r&b,soul,mexican pop,adult standards,nc hip hop,british soul,trap queen,hollywood,arkansas country,atl trap,underground hip hop,texas country,uk dance,house,new wave pop,brostep,dancehall,progressive house,funk,singer-songwriter,latin hip hop,idol,garage rock,mellow gold,baroque pop,big room,art pop,reggae fusion,cali rap,bronx hip hop,folk-pop,country rap,stomp and holler,neon pop punk,emo rap,punk,indie rock,funk rock,memphis hip hop,modern alternative rock,lgbtq+ hip hop,progressive electro house,alternative hip hop,blues rock,colombian pop,eurodance,classic rock,baton rouge rap,australian dance,folk,pop emo,soft rock,motown,pixie,canadian country,wrestling,glee club,complextro,vapor trap,etherpop,pittsburgh rap,escape room,indietronica,comic,german techno,new jersey rap,trap latino,houston rap,social media pop,puerto rican pop,deep southern trap,heartland rock,alternative dance,bubblegum dance,alberta country,outlaw country,country gospel,florida rap,hard rock,canadian metal,christian rock,soca,indiecoustica,harlem hip hop,new rave,electronic trap,christian music,grunge,show tunes,viral trap,la indie,swedish pop,swedish electropop,reggaeton flow,dance-punk,celtic rock,socal pop punk,lounge,chicano rap,stomp pop,ccm,vocal jazz,glam metal,worship,irish rock,electropowerpop,electro,indie pop rap,canadian contemporary country,bounce,christian alternative rock,south african rock,deep talent show,disco,hyphy,disco house,canadian latin,australian hip hop,nyc rap,brill building pop,k-pop,nz pop,minnesota hip hop,modern blues rock,album rock,modern folk rock,uk americana,old school hip hop,punk blues,dmv rap,industrial metal,skate punk,swedish synthpop,moombahton,Max_Peak_Position,Max_Rank_Change,In_Top5genres_Mean
count,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.00000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.00

In [21]:
df_clean_nogenre.describe()

,spotify_track_duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,Max_Peak_Position,Max_Rank_Change,In_Top5genres_Mean
count,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000,7858.000000
mean,225877.547340,0.634195,0.683831,5.258590,-5.955137,0.663019,0.110003,0.173873,0.008682,0.184399,0.506799,122.323709,3.972003,75.775261,12.531815,0.383706
std,46659.091277,0.149508,0.172011,3.589907,2.228035,0.472709,0.110535,0.215344,0.066326,0.140660,0.223494,29.558487,0.273062,24.561479,11.760207,0.466419
min,37013.000000,0.113000,0.031600,0.000000,-23.023000,0.000000,0.022400,0.000003,0.000000,0.020000,0.034900,48.718000,0.000000,1.000000,-8.000000,0.000000
25%,198219.250000,0.533250,0.571000,2.000000,-7.079000,0.000000,0.036200,0.019225,0.000000,0.095900,0.330000,97.943500,4.000000,66.000000,3.000000,0.000000
50%,221798.500000,0.638000,0.704000,5.000000,-5.640000,1.000000,0.057250,0.081900,0.000000,0.128000,0.505000,121.070000,4.000000,84.000000,11.000000,0.000000
75%,248459.500000,0.740000,0.819000,8.000000,-4.406500,1.000000,0.143000,0.247000,0.000017,0.234000,0.679000,142.397250,4.000000,95.000000,17.000000,1.000000
max,992160.000000,0.986000,0.996000,11.000000,0.175000,1.000000,0.951000,0.987000,0.982000,0.986000,0.976000,213.737000,5.000000,100.000000,79.000000,1.000000


### Features and Target Variables

I'm prepping 4 versions for XGBoost and k-NN:

1. Max Peak Position, no genre (nogenre__1 variables)
2. Max Peak Position, with genre (withgenre_1 variables)
3. Max Rank Change, no genre (nogenre_2 variables)
4. Max Rank Change, with genre (withgenre_2 variables)

In [22]:
# Prepare features and target for XGBoost and k-NN max_peak_position analysis, no genre
X_nogenre_1 = df_clean_nogenre.drop(['Max_Peak_Position'], axis=1)
y_nogenre_1 = df_clean_nogenre['Max_Peak_Position']

# Splitting the data into training and testing sets (75-25 split and random_state of 42)
X_nogenre_1_train, X_nogenre_1_test, y_nogenre_1_train, y_nogenre_1_test = train_test_split(X_nogenre_1, y_nogenre_1, test_size=0.25, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_nogenre_1_train_scaled = scaler.fit_transform(X_nogenre_1_train)
X_nogenre_1_test_scaled = scaler.fit_transform(X_nogenre_1_test)

In [23]:
# Prepare features and target for XGBoost and k-NN max_peak_position analysis, including genre
X_withgenre_1 = df_clean_withgenre.drop(['Max_Peak_Position'], axis=1)
y_withgenre_1 = df_clean_withgenre['Max_Peak_Position']

# Splitting the data into training and testing sets (75-25 split and random_state of 42)
X_withgenre_1_train, X_withgenre_1_test, y_withgenre_1_train, y_withgenre_1_test = train_test_split(X_withgenre_1, y_withgenre_1, test_size=0.25, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_withgenre_1_train_scaled = scaler.fit_transform(X_withgenre_1_train)
X_withgenre_1_test_scaled = scaler.fit_transform(X_withgenre_1_test)

In [24]:
# Prepare features and target for XGBoost and k-NN max_rank_change analysis, no genre
X_nogenre_2 = df_clean_nogenre.drop(['Max_Peak_Position', 'Max_Rank_Change'], axis=1)
y_nogenre_2 = df_clean_nogenre['Max_Rank_Change']

# Splitting the data into training and testing sets (75-25 split and random_state of 42)
X_nogenre_2_train, X_nogenre_2_test, y_nogenre_2_train, y_nogenre_2_test = train_test_split(X_nogenre_2, y_nogenre_2, test_size=0.25, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_nogenre_2_train_scaled = scaler.fit_transform(X_nogenre_2_train)
X_nogenre_2_test_scaled = scaler.fit_transform(X_nogenre_2_test)

In [25]:
# Prepare features and target for XGBoost and k-NN max_rank_change analysis, including genre
X_withgenre_2 = df_clean_withgenre.drop(['Max_Peak_Position', 'Max_Rank_Change'], axis=1)
y_withgenre_2 = df_clean_withgenre['Max_Rank_Change']

# Splitting the data into training and testing sets (75-25 split and random_state of 42)
X_withgenre_2_train, X_withgenre_2_test, y_withgenre_2_train, y_withgenre_2_test = train_test_split(X_withgenre_2, y_withgenre_2, test_size=0.25, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_withgenre_2_train_scaled = scaler.fit_transform(X_withgenre_2_train)
X_withgenre_2_test_scaled = scaler.fit_transform(X_withgenre_2_test)

Another 4 versions of the data for the deep learning model

1. Max Peak Position, no genre (nogenre_3 variables)
2. Max Peak Position, with genre (withgenre_3 variables)
3. Max Rank Change, no genre (nogenre_4 variables)
4. Max Rank Change, with genre (withgenre_4 variables)

In [26]:
# features and target for deep learning max_peak_position analysis, no genre
X_nogenre_3 = df_clean_nogenre.drop(['Max_Peak_Position'], axis=1)
y_nogenre_3 = df_clean_nogenre['Max_Peak_Position']

# Splitting the data into training and testing sets 
X_nogenre_3_train, X_nogenre_3_test, y_nogenre_3_train, y_nogenre_3_test = train_test_split(X_nogenre_3, y_nogenre_3, test_size=0.2, random_state=42)

# splitting training data into training and validiation
X_nogenre_3_train_final, X_nogenre_3_val, y_nogenre_3_train_final, y_nogenre_3_val = train_test_split(X_nogenre_3_train, y_nogenre_3_train, test_size=0.2, random_state=42)

# normalizing 
scaler.fit(X_nogenre_3_train_final)
X3_1_train_scaled = scaler.transform(X_nogenre_3_train_final)
X3_1_val_scaled = scaler.transform(X_nogenre_3_val)
X3_1_test_scaled = scaler.transform(X_nogenre_3_test)

In [27]:
# features and target for deep learning max_peak_position analysis, including genre
X_withgenre_3 = df_clean_withgenre.drop(['Max_Peak_Position'], axis=1)
y_withgenre_3 = df_clean_withgenre['Max_Peak_Position']

# Splitting the data into training and testing sets 
X_withgenre_3_train, X_withgenre_3_test, y_withgenre_3_train, y_withgenre_3_test = train_test_split(X_withgenre_3, y_withgenre_3, test_size=0.2, random_state=42)

# splitting training data into training and validiation
X_withgenre_3_train_final, X_withgenre_3_val, y_withgenre_3_train_final, y_withgenre_3_val = train_test_split(X_withgenre_3_train, y_withgenre_3_train, test_size=0.2, random_state=42)

# normalizing
scaler.fit(X_withgenre_3_train_final)
X3_2_train_scaled = scaler.transform(X_withgenre_3_train_final)
X3_2_val_scaled = scaler.transform(X_withgenre_3_val)
X3_2_test_scaled = scaler.transform(X_withgenre_3_test)

In [28]:
# features and target for deep learning max_rank_change analysis, no genre
X_nogenre_4 = df_clean_nogenre.drop(['Max_Peak_Position', 'Max_Rank_Change'], axis=1)
y_nogenre_4 = df_clean_nogenre['Max_Rank_Change']

# Splitting the data into training and testing sets 
X_nogenre_4_train, X_nogenre_4_test, y_nogenre_4_train, y_nogenre_4_test = train_test_split(X_nogenre_4, y_nogenre_4, test_size=0.2, random_state=42)

# splitting training data into training and validiation
X_nogenre_4_train_final, X_nogenre_4_val, y_nogenre_4_train_final, y_nogenre_4_val = train_test_split(X_nogenre_4_train, y_nogenre_4_train, test_size=0.2, random_state=42)

# normalizing
scaler.fit(X_nogenre_4_train_final)
X4_1_train_scaled = scaler.transform(X_nogenre_4_train_final)
X4_1_val_scaled = scaler.transform(X_nogenre_4_val)
X4_1_test_scaled = scaler.transform(X_nogenre_4_test)

In [29]:
# Prepare features and target for simple deep learning max_rank_change analysis, including genre
X_withgenre_4 = df_clean_withgenre.drop(['Max_Peak_Position', 'Max_Rank_Change'], axis=1)
y_withgenre_4 = df_clean_withgenre['Max_Rank_Change']

# Splitting the data into training and testing sets 
X_withgenre_4_train, X_withgenre_4_test, y_withgenre_4_train, y_withgenre_4_test = train_test_split(X_withgenre_4, y_withgenre_4, test_size=0.2, random_state=42)

# splitting training data into training and validiation
X_withgenre_4_train_final, X_withgenre_4_val, y_withgenre_4_train_final, y_withgenre_4_val = train_test_split(X_withgenre_4_train, y_withgenre_4_train, test_size=0.2, random_state=42)

# normalizing
scaler.fit(X_withgenre_4_train_final)
X4_2_train_scaled = scaler.transform(X_withgenre_4_train_final)
X4_2_val_scaled = scaler.transform(X_withgenre_4_val)
X4_2_test_scaled = scaler.transform(X_withgenre_4_test)

## Analysis

### XGBoost

**XGBoost | Max Peak Position - No Genre**

In [30]:
# XGBoost for max_peak_position, no genre

xgb_maxpeak_nogenre = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)

xgb_maxpeak_nogenre.fit(X_nogenre_1_train_scaled, y_nogenre_1_train)
y_nogenre_1_pred = xgb_maxpeak_nogenre.predict(X_nogenre_1_test_scaled)
y_nogenre_1_pred = np.clip(np.round(y_nogenre_1_pred), 1, 100)

rmse_nogenre_1 = np.sqrt(mean_squared_error(y_nogenre_1_test, y_nogenre_1_pred))
r2_nogenre_1 = r2_score(y_nogenre_1_test, y_nogenre_1_pred)

print(f'RMSE: {rmse_nogenre_1:.3f}')
print(f'R²: {r2_nogenre_1:.3f}')

RMSE: 25.189
R²: 0.009


In [31]:
# hyperparameter tuning 1

param_grid1 = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

grid_search_xgb_nogenre_1 = GridSearchCV(estimator=xgb_maxpeak_nogenre,
                            param_grid=param_grid1,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb_nogenre_1.fit(X_nogenre_1_train, y_nogenre_1_train)

print("Best parameters:", grid_search_xgb_nogenre_1.best_params_)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 3, 'subsample': 0.8}


In [32]:
# hyperparameter tuning 2

param_grid2 = {
    'max_depth': [2, 3, 4],
    'learning_rate': [0.05, 0.1, 0.15,],
    'subsample': [0.75, 0.8, 0.85],
    'colsample_bytree': [0.9, 1.0, 1.1]
}

grid_search_xgb_nogenre_2 = GridSearchCV(estimator=xgb_maxpeak_nogenre,
                            param_grid=param_grid2,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb_nogenre_2.fit(X_nogenre_1_train, y_nogenre_1_train)

print("Best parameters:", grid_search_xgb_nogenre_2.best_params_)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
Best parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 2, 'subsample': 0.85}


c:\Users\marha\.conda\envs\ai-environment\lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
135 fits failed out of a total of 405.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
135 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\marha\.conda\envs\ai-environment\lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\marha\.conda\envs\ai-environment\lib\site-packages\xgboost\core.py", line 726, in inner_f
    return func(**kwargs)
  File "c:\Users\marha\.conda\envs\ai-environment\lib\site-packages\xgboost\sklearn.py", line 1170, in fit
    self._Booster = train(
  File

In [33]:
# hyperparameter tuning 3

param_grid3 = {
    'max_depth': [2],
    'learning_rate': [0.07, 0.1, 0.13],
    'subsample': [0.85],
    'colsample_bytree': [1.0],
}

grid_search_xgb_nogenre_3 = GridSearchCV(estimator=xgb_maxpeak_nogenre,
                            param_grid=param_grid3,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb_nogenre_3.fit(X_nogenre_1_train, y_nogenre_1_train)

print("Best parameters:", grid_search_xgb_nogenre_3.best_params_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 2, 'subsample': 0.85}


In [34]:
# Extract best/final model for max_peak_position
best_xgb1_1 = grid_search_xgb_nogenre_3.best_estimator_

# predictions
y_nogenre_1_pred_best = best_xgb1_1.predict(X_nogenre_1_test)
y_nogenre_1_pred_best = np.clip(np.round(y_nogenre_1_pred_best), 1, 100)

# Evaluate XGBoost model
rmse_nogenre_1_best = np.sqrt(mean_squared_error(y_nogenre_1_test, y_nogenre_1_pred_best))
r2_nogenre_1_best = r2_score(y_nogenre_1_test, y_nogenre_1_pred_best)

print(f'RMSE: {rmse_nogenre_1_best:.3f}')
print(f'R²: {r2_nogenre_1_best:.3f}')

RMSE: 22.851
R²: 0.185


**XGBoost | Max Peak Position - With Genre**

In [35]:
# XGBoost for max_peak_position, with genre

xgb_maxpeak_withgenre = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)

xgb_maxpeak_withgenre.fit(X_withgenre_1_train, y_withgenre_1_train)
y_withgenre_1_pred = xgb_maxpeak_withgenre.predict(X_withgenre_1_test)
y_withgenre_1_pred = np.clip(np.round(y_withgenre_1_pred), 1, 100)

rmse_withgenre_1 = np.sqrt(mean_squared_error(y_withgenre_1_test, y_withgenre_1_pred))
r2_withgenre_1 = r2_score(y_withgenre_1_test, y_withgenre_1_pred)

print(f'RMSE: {rmse_withgenre_1:.3f}')
print(f'R²: {r2_withgenre_1:.3f}')

RMSE: 22.751
R²: 0.192


In [36]:
# hyperparameter tuning 1

param_grid1 = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

grid_search_xgb_withgenre_1 = GridSearchCV(estimator=xgb_maxpeak_withgenre,
                            param_grid=param_grid1,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb_withgenre_1.fit(X_withgenre_1_train, y_withgenre_1_train)

print("Best parameters:", grid_search_xgb_withgenre_1.best_params_)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 3, 'subsample': 1.0}


In [37]:
# hyperparameter tuning 2

param_grid4 = {
    'max_depth': [2, 3, 4],
    'learning_rate': [0.15, 0.2, 0.25],
    'subsample': [0.9, 1.0, 1.1],
    'colsample_bytree': [0.6, 0.7, 0.8],
}

grid_search_xgb_withgenre_2 = GridSearchCV(estimator=xgb_maxpeak_withgenre,
                            param_grid=param_grid4,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb_withgenre_2.fit(X_withgenre_1_train, y_withgenre_1_train)

print("Best parameters:", grid_search_xgb_withgenre_2.best_params_)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


c:\Users\marha\.conda\envs\ai-environment\lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
135 fits failed out of a total of 405.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
135 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\marha\.conda\envs\ai-environment\lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\marha\.conda\envs\ai-environment\lib\site-packages\xgboost\core.py", line 726, in inner_f
    return func(**kwargs)
  File "c:\Users\marha\.conda\envs\ai-environment\lib\site-packages\xgboost\sklearn.py", line 1170, in fit
    self._Booster = train(
  File

Best parameters: {'colsample_bytree': 0.6, 'learning_rate': 0.15, 'max_depth': 3, 'subsample': 1.0}


In [38]:
# hyperparameter tuning 3

param_grid5 = {
    'max_depth': [3],
    'learning_rate': [0.12, 0.15, 0.17],
    'subsample': [1.0],
    'colsample_bytree': [0.5, 0.6],
}

grid_search_xgb_withgenre_3 = GridSearchCV(estimator=xgb_maxpeak_withgenre,
                            param_grid=param_grid5,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb_withgenre_3.fit(X_withgenre_1_train, y_withgenre_1_train)

print("Best parameters:", grid_search_xgb_withgenre_3.best_params_)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Best parameters: {'colsample_bytree': 0.6, 'learning_rate': 0.15, 'max_depth': 3, 'subsample': 1.0}


In [39]:
# Extract best/final model 
best_xgb_maxpeak_withgenre = grid_search_xgb_withgenre_3.best_estimator_

# predictions
y_withgenre_1_pred_best = best_xgb_maxpeak_withgenre.predict(X_withgenre_1_test)
y_withgenre_1_pred_best = np.clip(np.round(y_withgenre_1_pred_best), 1, 100)

# Evaluate XGBoost model
rmse_withgenre_2_best = np.sqrt(mean_squared_error(y_withgenre_1_test, y_withgenre_1_pred_best))
r2_withgenre_2_best = r2_score(y_withgenre_1_test, y_withgenre_1_pred_best)

print(f'RMSE: {rmse_withgenre_2_best:.3f}')
print(f'R²: {r2_withgenre_2_best:.3f}')

RMSE: 21.607
R²: 0.271


**XGBoost | Max Rank Change - No Genre**

In [40]:
# XGBoost for max_rank_change

xgb_maxrank_nogenre = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)

xgb_maxrank_nogenre.fit(X_nogenre_2_train, y_nogenre_2_train)
y_nogenre_2_pred = xgb_maxrank_nogenre.predict(X_nogenre_2_test)
y_nogenre_2_pred = np.clip(np.round(y_nogenre_2_pred), 1, 100)

rmse_maxrank_nogenre_2 = np.sqrt(mean_squared_error(y_nogenre_2_test, y_nogenre_2_pred))
r2_maxrank_nogenre_2 = r2_score(y_nogenre_2_test, y_nogenre_2_pred)

print(f'RMSE: {rmse_maxrank_nogenre_2:.3f}')
print(f'R²: {r2_maxrank_nogenre_2:.3f}')

RMSE: 12.605
R²: -0.097


In [41]:
# hyperparameter tuning 1 for max_rank_change

param_grid1 = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

grid_search_xgb_maxrank_nogenre_1 = GridSearchCV(estimator=xgb_maxrank_nogenre,
                            param_grid=param_grid1,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb_maxrank_nogenre_1.fit(X_nogenre_2_train, y_nogenre_2_train)

print("Best parameters:", grid_search_xgb_maxrank_nogenre_1.best_params_)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 3, 'subsample': 1.0}


In [43]:
# hyperparameter tuning 2 for max_rank_change

param_grid4 = {
    'max_depth': [2, 3, 4],
    'learning_rate': [0.005, 0.01, 0.015],
    'subsample': [1.0, 1.1],
    'colsample_bytree': [1.0, 1.1],
}

grid_search_xgb_maxrank_nogenre_2 = GridSearchCV(estimator=xgb_maxrank_nogenre,
                            param_grid=param_grid4,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb_maxrank_nogenre_2.fit(X_nogenre_2_train, y_nogenre_2_train)

print("Best parameters:", grid_search_xgb_maxrank_nogenre_2.best_params_)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.015, 'max_depth': 3, 'subsample': 1.0}


c:\Users\marha\.conda\envs\ai-environment\lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
135 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
45 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\marha\.conda\envs\ai-environment\lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\marha\.conda\envs\ai-environment\lib\site-packages\xgboost\core.py", line 726, in inner_f
    return func(**kwargs)
  File "c:\Users\marha\.conda\envs\ai-environment\lib\site-packages\xgboost\sklearn.py", line 1170, in fit
    self._Booster = train(
  File 

In [44]:
# Extract best/final model  for max_rank_change
best_xgb_maxrank_nogenre = grid_search_xgb_maxrank_nogenre_2.best_estimator_

y_nogenre_2_pred_best = best_xgb_maxrank_nogenre.predict(X_nogenre_2_test)
y_nogenre_2_pred_best = np.clip(np.round(y_nogenre_2_pred_best), 1, 100)

# Evaluate XGBoost model
rmse_maxrank_nogenre_2_best = np.sqrt(mean_squared_error(y_nogenre_2_test, y_nogenre_2_pred_best))
r2_maxrank_nogenre_2_best = r2_score(y_nogenre_2_test, y_nogenre_2_pred_best)

print(f'RMSE: {rmse_maxrank_nogenre_2_best:.3f}')
print(f'R²: {r2_maxrank_nogenre_2_best:.3f}')

RMSE: 11.918
R²: 0.020


**XGBoost | Max Rank Change - With Genre**

In [45]:
# XGBoost for max_rank_change

xgb_maxrank_withgenre = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)

xgb_maxrank_withgenre.fit(X_withgenre_2_train, y_withgenre_2_train)
y_withgenre_2_pred = xgb_maxrank_withgenre.predict(X_withgenre_2_test)
y_withgenre_2_pred = np.clip(np.round(y_withgenre_2_pred), 1, 100)

rmse_maxpeak_withgenre_1 = np.sqrt(mean_squared_error(y_withgenre_2_test, y_withgenre_2_pred))
r2_maxpeak_withgenre_1 = r2_score(y_withgenre_2_test, y_withgenre_2_pred)

print(f'RMSE: {rmse_maxpeak_withgenre_1:.3f}')
print(f'R²: {r2_maxpeak_withgenre_1:.3f}')

RMSE: 12.342
R²: -0.051


In [46]:
# hyperparameter tuning 1

param_grid1 = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

grid_search_xgb_maxrank_withgenre_1 = GridSearchCV(estimator=xgb_maxrank_withgenre,
                            param_grid=param_grid1,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb_maxrank_withgenre_1.fit(X_withgenre_2_train, y_withgenre_2_train)

print("Best parameters:", grid_search_xgb_maxrank_withgenre_1.best_params_)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 6, 'subsample': 0.8}


In [47]:
# hyperparameter tuning 2

param_grid6 = {
    'max_depth': [5, 6, 7],
    'learning_rate': [0.005, 0.01, 0.015],
    'subsample': [0.7, 0.8],
    'colsample_bytree': [1.0, 1.1],
}

grid_search_xgb_maxrank_withgenre_2 = GridSearchCV(estimator=xgb_maxrank_withgenre,
                            param_grid=param_grid6,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb_maxrank_withgenre_2.fit(X_withgenre_2_train, y_withgenre_2_train)

print("Best parameters:", grid_search_xgb_maxrank_withgenre_2.best_params_)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


c:\Users\marha\.conda\envs\ai-environment\lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
90 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\marha\.conda\envs\ai-environment\lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\marha\.conda\envs\ai-environment\lib\site-packages\xgboost\core.py", line 726, in inner_f
    return func(**kwargs)
  File "c:\Users\marha\.conda\envs\ai-environment\lib\site-packages\xgboost\sklearn.py", line 1170, in fit
    self._Booster = train(
  File "

Best parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.015, 'max_depth': 6, 'subsample': 0.7}


In [48]:
# hyperparameter tuning 3

param_grid7 = {
    'max_depth': [6],
    'learning_rate': [0.013, 0.015, 0.017],
    'subsample': [0.5, 0.6, 0.7],
    'colsample_bytree': [1.0],
}

grid_search_xgb_maxrank_withgenre_3 = GridSearchCV(estimator=xgb_maxrank_withgenre,
                            param_grid=param_grid7,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb_maxrank_withgenre_3.fit(X_withgenre_2_train, y_withgenre_2_train)

print("Best parameters:", grid_search_xgb_maxrank_withgenre_3.best_params_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.015, 'max_depth': 6, 'subsample': 0.7}


In [49]:
# Extract best/final model  for max_rank_change
best_xgb_maxrank_withgenre = grid_search_xgb_maxrank_withgenre_3.best_estimator_

y_withgenre_2_pred_best = best_xgb_maxrank_withgenre.predict(X_withgenre_2_test)
y_withgenre_2_pred_best = np.clip(np.round(y_withgenre_2_pred_best), 1, 100)

# Evaluate XGBoost model
rmse_maxpeak_withgenre_2_best = np.sqrt(mean_squared_error(y_withgenre_2_test, y_withgenre_2_pred_best))
r2_maxpeak_withgenre_2_best = r2_score(y_withgenre_2_test, y_withgenre_2_pred_best)

print(f'RMSE: {rmse_maxpeak_withgenre_2_best:.3f}')
print(f'R²: {r2_maxpeak_withgenre_2_best:.3f}')

RMSE: 11.764
R²: 0.045


#### XGBoost Summary

Using XGBoost, including the genre features slightly improved model performance. However, the Max Rank Change models both had an r<sup>2</sup> value less than 0.001, essentially indicating no fit of the model to the test data. Max Peak Position performed better, but the highest r<sup>2</sup> value was 0.271 so their predictive value is low.

Given the lack of predictive power in these outcomes, I'm shifting focus to the other two techniques.

### k-Nearest Neighbors

**k-Nearest Neighbors | Max Peak Position - No Genre**

In [ ]:
# Define parameter grid for standard metrics
param_grid8 = {
    'n_neighbors': list(range(1, 31, 2)),  # Odd values to avoid ties
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'chebyshev'],
    'algorithm': ['auto']
}

# Create standard KNN model for grid search
knn = KNeighborsClassifier()

# Perform grid search
print("Starting grid search for standard metrics...")
grid_search1_1 = GridSearchCV(knn, param_grid8, cv=5, scoring='accuracy', n_jobs=-1)
grid_search1_1.fit(X1_1_train_scaled, y1_1_train)

# Get best parameters and score
standard_best_params1_1 = grid_search1_1.best_params_
standard_best_score1_1 = grid_search1_1.best_score_
print(f"Best parameters (standard metrics): {standard_best_params1_1}")
print(f"Best cross-validation accuracy: {standard_best_score1_1:.4f}")


In [ ]:
# final model with best parameters
final_model1_1 = grid_search1_1.best_estimator_

# predictions on test set
y1_1_pred = final_model1_1.predict(X1_1_test_scaled)

# Evaluate the model
accuracy1_1 = accuracy_score(y1_1_test, y1_1_pred)

print(f"Best accuracy: {accuracy1_1:.4f}")

**k-Nearest Neighbors | Max Peak Position - With Genre**

In [ ]:
# arameter grid for standard metrics
param_grid8 = {
    'n_neighbors': list(range(1, 31, 2)),  # Odd values to avoid ties
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'chebyshev'],
    'algorithm': ['auto']
}

# Create standard KNN model for grid search
knn = KNeighborsClassifier()

# Perform grid search
print("Starting grid search for standard metrics...")
grid_search1_2 = GridSearchCV(knn, param_grid8, cv=5, scoring='accuracy', n_jobs=-1)
grid_search1_2.fit(X1_2_train_scaled, y1_2_train)

# Get best parameters and score
standard_best_params1_2 = grid_search1_2.best_params_
standard_best_score1_2 = grid_search1_2.best_score_
print(f"Best parameters (standard metrics): {standard_best_params1_2}")
print(f"Best cross-validation accuracy: {standard_best_score1_2:.4f}")


In [ ]:
# final model with best parameters
final_model1_2 = grid_search1_2.best_estimator_

# predictions on test set
y1_2_pred = final_model1_2.predict(X1_2_test_scaled)

# Evaluate the model
accuracy1_2 = accuracy_score(y1_2_test, y1_2_pred)

print(f"Best accuracy: {accuracy1_2:.4f}")

**k-Nearest Neighbors | Max Rank Change - No Genre**

In [ ]:
# Define parameter grid for standard metrics
param_grid8 = {
    'n_neighbors': list(range(1, 31, 2)),  # Odd values to avoid ties
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'chebyshev'],
    'algorithm': ['auto']
}

# Create standard KNN model for grid search
knn = KNeighborsClassifier()

# Perform grid search
print("Starting grid search for standard metrics...")
grid_search2_1 = GridSearchCV(knn, param_grid8, cv=5, scoring='accuracy', n_jobs=-1)
grid_search2_1.fit(X2_1_train_scaled, y2_1_train)

# Get best parameters and score
standard_best_params2_1 = grid_search2_1.best_params_
standard_best_score2_1 = grid_search2_1.best_score_
print(f"Best parameters (standard metrics): {standard_best_params2_1}")
print(f"Best cross-validation accuracy: {standard_best_score2_1:.4f}")


In [ ]:
# final model with best parameters
final_model2_1 = grid_search2_1.best_estimator_


# Make predictions on test set
y2_1_pred = final_model2_1.predict(X2_1_test_scaled)


# Evaluate the model
accuracy2_1 = accuracy_score(y2_1_test, y2_1_pred)

print(f"Best accuracy: {accuracy2_1:.4f}")

**k-Nearest Neighbors | Max Rank Change - With Genre**

In [ ]:
# Define parameter grid for standard metrics
param_grid8 = {
    'n_neighbors': list(range(1, 31, 2)),  # Odd values to avoid ties
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'chebyshev'],
    'algorithm': ['auto']
}

# Create standard KNN model for grid search
knn = KNeighborsClassifier()

# Perform grid search
print("Starting grid search for standard metrics...")
grid_search2_2 = GridSearchCV(knn, param_grid8, cv=5, scoring='accuracy', n_jobs=-1)
grid_search2_2.fit(X2_2_train_scaled, y2_2_train)

# Get best parameters and score
standard_best_params2_2 = grid_search2_2.best_params_
standard_best_score2_2 = grid_search2_2.best_score_
print(f"Best parameters (standard metrics): {standard_best_params2_2}")
print(f"Best cross-validation accuracy: {standard_best_score2_2:.4f}")


In [ ]:
# final model with best parameters
final_model2_2 = grid_search2_2.best_estimator_


# Make predictions on test set
y2_2_pred = final_model2_2.predict(X2_2_test_scaled)


# Evaluate the model
accuracy2_2 = accuracy_score(y2_2_test, y2_2_pred)

print(f"Best accuracy: {accuracy2_2:.4f}")

#### k-NN Summary

The k-NN models performed poorly on the Max Peak Position data, with a maximum accuracy of 0.052. The performance on the Max Rank Change was better, but the maximum accuracy was still just 0.217.

I will not explore k-NN further and instead focus on the deep learning model.

### Deep Learning

**Deep Learning | Max Peak Position - No Genre**

In [ ]:
# deep learning model, no regularization or dropout

baseline_model3_1 = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X3_1_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # Single output for regression
])

baseline_model3_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_baseline3_1 = baseline_model3_1.fit(
    X3_1_train_scaled, y3_1_train_final,
    validation_data=(X3_1_val_scaled, y3_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# deep learning model with batch normalization

bnorm_model3_1 = keras.Sequential([
    layers.Dense(64, activation='linear', input_shape=(X3_1_train.shape[1],)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    
    layers.Dense(64, activation='linear'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    
    layers.Dense(64, activation='linear'),
    layers.BatchNormalization(),
    layers.Activation('relu'),

    layers.Dense(1)  # Single output for regression
])

bnorm_model3_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_bnorm_model3_1 = bnorm_model3_1.fit(
    X3_1_train_scaled, y3_1_train_final,
    validation_data=(X3_1_val_scaled, y3_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# deep learning model regularization (L2 and dropout)

l2_reg = 1e-4
dropout_rate = 0.4

reg_model3_1 = keras.Sequential([
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg),
                 input_shape=(X3_1_train.shape[1],)),
    layers.Dropout(dropout_rate),
        
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(1)  # Single output for regression
])

reg_model3_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_reg_model3_1 = reg_model3_1.fit(
    X3_1_train_scaled, y3_1_train_final,
    validation_data=(X3_1_val_scaled, y3_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# model evaluation
print("MODEL EVALUATION: MAX PEAK POSITION, NO GENRE")

print("\n=== Baseline Model ===")
train_scores3_1 = baseline_model3_1.evaluate(X3_1_train_scaled, y3_1_train_final, verbose=0)
val_scores3_1   = baseline_model3_1.evaluate(X3_1_val_scaled, y3_1_val, verbose=0)
print(f"Train MAE: {train_scores3_1[1]:.4f}, Train MSE: {train_scores3_1[2]:.4f}")
print(f"Val   MAE: {val_scores3_1[1]:.4f}, Val   MSE: {val_scores3_1[2]:.4f}")

print("\n=== BatchNorm Model ===")
train_scores_bn3_1 = bnorm_model3_1.evaluate(X3_1_train_scaled, y3_1_train_final, verbose=0)
val_scores_bn3_1   = bnorm_model3_1.evaluate(X3_1_val_scaled, y3_1_val, verbose=0)
print(f"Train MAE: {train_scores_bn3_1[1]:.4f}, Train MSE: {train_scores_bn3_1[2]:.4f}")
print(f"Val   MAE: {val_scores_bn3_1[1]:.4f}, Val   MSE: {val_scores_bn3_1[2]:.4f}")

print("\n=== Regularized Model (L2 + Dropout) ===")
train_scores_reg3_1 = reg_model3_1.evaluate(X3_1_train_scaled, y3_1_train_final, verbose=0)
val_scores_reg3_1   = reg_model3_1.evaluate(X3_1_val_scaled, y3_1_val, verbose=0)
print(f"Train MAE: {train_scores_reg3_1[1]:.4f}, Train MSE: {train_scores_reg3_1[2]:.4f}")
print(f"Val   MAE: {val_scores_reg3_1[1]:.4f}, Val   MSE: {val_scores_reg3_1[2]:.4f}")


**Deep Learning | Max Peak Position - With Genre**

In [ ]:
# deep learning model, no regularization or dropout

baseline_model3_2 = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X3_2_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # Single output for regression
])

baseline_model3_2.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_baseline3_2 = baseline_model3_2.fit(
    X3_2_train_scaled, y3_2_train_final,
    validation_data=(X3_2_val_scaled, y3_2_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# deep learning model with batch normalization

bnorm_model3_2 = keras.Sequential([
    layers.Dense(64, activation='linear', input_shape=(X3_2_train.shape[1],)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    
    layers.Dense(64, activation='linear'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    
    layers.Dense(64, activation='linear'),
    layers.BatchNormalization(),
    layers.Activation('relu'),

    layers.Dense(1)  # Single output for regression
])

bnorm_model3_2.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_bnorm_model3_2 = bnorm_model3_2.fit(
    X3_2_train_scaled, y3_2_train_final,
    validation_data=(X3_2_val_scaled, y3_2_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# deep learning model with regularization (L2 and dropout)

l2_reg = 1e-4
dropout_rate = 0.4

reg_model3_2 = keras.Sequential([
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg),
                 input_shape=(X3_2_train.shape[1],)),
    layers.Dropout(dropout_rate),
        
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(1)  # Single output for regression
])

reg_model3_2.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_reg_model3_2 = reg_model3_2.fit(
    X3_2_train_scaled, y3_2_train_final,
    validation_data=(X3_2_val_scaled, y3_2_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# model evaluation
print("MODEL EVALUATION: MAX PEAK POSITION, WITH GENRE")

print("\n=== Baseline Model ===")
train_scores3_2 = baseline_model3_2.evaluate(X3_2_train_scaled, y3_2_train_final, verbose=0)
val_scores3_2   = baseline_model3_2.evaluate(X3_2_val_scaled, y3_2_val, verbose=0)
print(f"Train MAE: {train_scores3_2[1]:.4f}, Train MSE: {train_scores3_2[2]:.4f}")
print(f"Val   MAE: {val_scores3_2[1]:.4f}, Val   MSE: {val_scores3_2[2]:.4f}")

print("\n=== BatchNorm Model ===")
train_scores_bn3_2 = bnorm_model3_2.evaluate(X3_2_train_scaled, y3_2_train_final, verbose=0)
val_scores_bn3_2   = bnorm_model3_2.evaluate(X3_2_val_scaled, y3_2_val, verbose=0)
print(f"Train MAE: {train_scores_bn3_2[1]:.4f}, Train MSE: {train_scores_bn3_2[2]:.4f}")
print(f"Val   MAE: {val_scores_bn3_2[1]:.4f}, Val   MSE: {val_scores_bn3_2[2]:.4f}")

print("\n=== Regularized Model (L2 + Dropout) ===")
train_scores_reg3_2 = reg_model3_2.evaluate(X3_2_train_scaled, y3_2_train_final, verbose=0)
val_scores_reg3_2   = reg_model3_2.evaluate(X3_2_val_scaled, y3_2_val, verbose=0)
print(f"Train MAE: {train_scores_reg3_2[1]:.4f}, Train MSE: {train_scores_reg3_2[2]:.4f}")
print(f"Val   MAE: {val_scores_reg3_2[1]:.4f}, Val   MSE: {val_scores_reg3_2[2]:.4f}")


**Deep Learning | Max Rank Change - No Genre**

In [ ]:
# deep learning model, no regularization or dropout

baseline_model4_1 = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X4_1_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # Single output for regression
])

baseline_model4_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_baseline4_1 = baseline_model4_1.fit(
    X4_1_train_scaled, y4_1_train_final,
    validation_data=(X4_1_val_scaled, y4_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# deep learning model with batch normalization

bnorm_model4_1 = keras.Sequential([
    layers.Dense(64, activation='linear', input_shape=(X4_1_train.shape[1],)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    
    layers.Dense(64, activation='linear'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    
    layers.Dense(64, activation='linear'),
    layers.BatchNormalization(),
    layers.Activation('relu'),

    layers.Dense(1)  # Single output for regression
])

bnorm_model4_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_bnorm_model4_1 = bnorm_model4_1.fit(
    X4_1_train_scaled, y4_1_train_final,
    validation_data=(X4_1_val_scaled, y4_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# deep learning model with regularization (L2 and dropout)

l2_reg = 1e-4
dropout_rate = 0.4

reg_model4_1 = keras.Sequential([
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg),
                 input_shape=(X4_1_train.shape[1],)),
    layers.Dropout(dropout_rate),
        
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(1)  # Single output for regression
])

reg_model4_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_reg_model4_1 = reg_model4_1.fit(
    X4_1_train_scaled, y4_1_train_final,
    validation_data=(X4_1_val_scaled, y4_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# model evaluation
print("MODEL EVALUATION: MAX RANK CHANGE, NO GENRE")

print("\n=== Baseline Model ===")
train_scores4_1 = baseline_model4_1.evaluate(X4_1_train_scaled, y4_1_train_final, verbose=0)
val_scores4_1   = baseline_model4_1.evaluate(X4_1_val_scaled, y4_1_val, verbose=0)
print(f"Train MAE: {train_scores4_1[1]:.4f}, Train MSE: {train_scores4_1[2]:.4f}")
print(f"Val   MAE: {val_scores4_1[1]:.4f}, Val   MSE: {val_scores4_1[2]:.4f}")

print("\n=== BatchNorm Model ===")
train_scores_bn4_1 = bnorm_model4_1.evaluate(X4_1_train_scaled, y4_1_train_final, verbose=0)
val_scores_bn4_1   = bnorm_model4_1.evaluate(X4_1_val_scaled, y4_1_val, verbose=0)
print(f"Train MAE: {train_scores_bn4_1[1]:.4f}, Train MSE: {train_scores_bn4_1[2]:.4f}")
print(f"Val   MAE: {val_scores_bn4_1[1]:.4f}, Val   MSE: {val_scores_bn4_1[2]:.4f}")

print("\n=== Regularized Model (L2 + Dropout) ===")
train_scores_reg4_1 = reg_model4_1.evaluate(X4_1_train_scaled, y4_1_train_final, verbose=0)
val_scores_reg4_1   = reg_model4_1.evaluate(X4_1_val_scaled, y4_1_val, verbose=0)
print(f"Train MAE: {train_scores_reg4_1[1]:.4f}, Train MSE: {train_scores_reg4_1[2]:.4f}")
print(f"Val   MAE: {val_scores_reg4_1[1]:.4f}, Val   MSE: {val_scores_reg4_1[2]:.4f}")


**Deep Learning | Max Rank Change - With Genre**

In [ ]:
# deep learning model, no regularization or dropout

baseline_model4_2 = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X4_2_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # Single output for regression
])

baseline_model4_2.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_baseline4_2 = baseline_model4_2.fit(
    X4_2_train_scaled, y4_2_train_final,
    validation_data=(X4_2_val_scaled, y4_2_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# deep learning model with batch normalization

bnorm_model4_2 = keras.Sequential([
    layers.Dense(64, activation='linear', input_shape=(X4_2_train.shape[1],)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    
    layers.Dense(64, activation='linear'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    
    layers.Dense(64, activation='linear'),
    layers.BatchNormalization(),
    layers.Activation('relu'),

    layers.Dense(1)  # Single output for regression
])

bnorm_model4_2.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_bnorm_model4_2 = bnorm_model4_2.fit(
    X4_2_train_scaled, y4_2_train_final,
    validation_data=(X4_2_val_scaled, y4_2_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# deep learning model with regularization (L2 and dropout)

l2_reg = 1e-4
dropout_rate = 0.4

reg_model4_2 = keras.Sequential([
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg),
                 input_shape=(X4_2_train.shape[1],)),
    layers.Dropout(dropout_rate),
        
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(1)  # Single output for regression
])

reg_model4_2.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_reg_model4_2 = reg_model4_2.fit(
    X4_2_train_scaled, y4_2_train_final,
    validation_data=(X4_2_val_scaled, y4_2_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# model evaluation
print("MODEL EVALUATION: MAX RANK CHANGE, WITH GENRE")

print("\n=== Baseline Model ===")
train_scores4_2 = baseline_model4_2.evaluate(X4_2_train_scaled, y4_2_train_final, verbose=0)
val_scores4_2   = baseline_model4_2.evaluate(X4_2_val_scaled, y4_2_val, verbose=0)
print(f"Train MAE: {train_scores4_2[1]:.4f}, Train MSE: {train_scores4_2[2]:.4f}")
print(f"Val   MAE: {val_scores4_2[1]:.4f}, Val   MSE: {val_scores4_2[2]:.4f}")

print("\n=== BatchNorm Model ===")
train_scores_bn4_2 = bnorm_model4_2.evaluate(X4_2_train_scaled, y4_2_train_final, verbose=0)
val_scores_bn4_2   = bnorm_model4_2.evaluate(X4_2_val_scaled, y4_2_val, verbose=0)
print(f"Train MAE: {train_scores_bn4_1[1]:.4f}, Train MSE: {train_scores_bn4_1[2]:.4f}")
print(f"Val   MAE: {val_scores_bn4_1[1]:.4f}, Val   MSE: {val_scores_bn4_1[2]:.4f}")

print("\n=== Regularized Model (L2 + Dropout) ===")
train_scores_reg4_2 = reg_model4_2.evaluate(X4_2_train_scaled, y4_2_train_final, verbose=0)
val_scores_reg4_2   = reg_model4_2.evaluate(X4_2_val_scaled, y4_2_val, verbose=0)
print(f"Train MAE: {train_scores_reg4_2[1]:.4f}, Train MSE: {train_scores_reg4_2[2]:.4f}")
print(f"Val   MAE: {val_scores_reg4_2[1]:.4f}, Val   MSE: {val_scores_reg4_2[2]:.4f}")


**Deep Learning | Optimizing Regularized Models**

Max Peak Position

In [ ]:
# adding a layer

l2_reg = 1e-4
dropout_rate = 0.4

reg_model3_1 = keras.Sequential([
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg),
                 input_shape=(X3_1_train.shape[1],)),
    layers.Dropout(dropout_rate),
        
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(1)  # Single output for regression
])

reg_model3_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_reg_model3_1 = reg_model3_1.fit(
    X3_1_train_scaled, y3_1_train_final,
    validation_data=(X3_1_val_scaled, y3_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
print("\n=== Regularized Model (L2 + Dropout) ===")
print("    Adding an additional deep layer")
train_scores_reg3_1 = reg_model3_1.evaluate(X3_1_train_scaled, y3_1_train_final, verbose=0)
val_scores_reg3_1   = reg_model3_1.evaluate(X3_1_val_scaled, y3_1_val, verbose=0)
print(f"Train MAE: {train_scores_reg3_1[1]:.4f}, Train MSE: {train_scores_reg3_1[2]:.4f}")
print(f"Val   MAE: {val_scores_reg3_1[1]:.4f}, Val   MSE: {val_scores_reg3_1[2]:.4f}")


Adding a layer resulted in a larger MAE for both training and validation. Removing the additional layer and adding kernels.

In [ ]:
# adding a layer

l2_reg = 1e-4
dropout_rate = 0.4

reg_model3_1 = keras.Sequential([
    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg),
                 input_shape=(X3_1_train.shape[1],)),
    layers.Dropout(dropout_rate),
        
    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(1)  # Single output for regression
])

reg_model3_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_reg_model3_1 = reg_model3_1.fit(
    X3_1_train_scaled, y3_1_train_final,
    validation_data=(X3_1_val_scaled, y3_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
print("\n=== Regularized Model (L2 + Dropout) ===")
print("Reverting to 2 deep layers, increasing to 128 kernels per layer")
train_scores_reg3_1 = reg_model3_1.evaluate(X3_1_train_scaled, y3_1_train_final, verbose=0)
val_scores_reg3_1   = reg_model3_1.evaluate(X3_1_val_scaled, y3_1_val, verbose=0)
print(f"Train MAE: {train_scores_reg3_1[1]:.4f}, Train MSE: {train_scores_reg3_1[2]:.4f}")
print(f"Val   MAE: {val_scores_reg3_1[1]:.4f}, Val   MSE: {val_scores_reg3_1[2]:.4f}")


Increasing kernels per layer gave a result very similar to the original model but increased overfitting a little bit. Leaving 128 kernels per layer and increasing the dropout rate.

In [ ]:
# increasing dropout rate

l2_reg = 1e-4
dropout_rate = 0.6

reg_model3_1 = keras.Sequential([
    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg),
                 input_shape=(X3_1_train.shape[1],)),
    layers.Dropout(dropout_rate),
        
    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(1)  # Single output for regression
])

reg_model3_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_reg_model3_1 = reg_model3_1.fit(
    X3_1_train_scaled, y3_1_train_final,
    validation_data=(X3_1_val_scaled, y3_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
print("\n=== Regularized Model (L2 + Dropout) ===")
print("2 deep layers,  kernels per layer, increased dropout from 0.4 to 0.6")
train_scores_reg3_1 = reg_model3_1.evaluate(X3_1_train_scaled, y3_1_train_final, verbose=0)
val_scores_reg3_1   = reg_model3_1.evaluate(X3_1_val_scaled, y3_1_val, verbose=0)
print(f"Train MAE: {train_scores_reg3_1[1]:.4f}, Train MSE: {train_scores_reg3_1[2]:.4f}")
print(f"Val   MAE: {val_scores_reg3_1[1]:.4f}, Val   MSE: {val_scores_reg3_1[2]:.4f}")


In [ ]:
# adding a layer

l2_reg = 1e-4
dropout_rate = 0.6

reg_model3_1 = keras.Sequential([
    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg),
                 input_shape=(X3_1_train.shape[1],)),
    layers.Dropout(dropout_rate),
        
    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(1)  # Single output for regression
])

reg_model3_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_reg_model3_1 = reg_model3_1.fit(
    X3_1_train_scaled, y3_1_train_final,
    validation_data=(X3_1_val_scaled, y3_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
print("\n=== Regularized Model (L2 + Dropout) ===")
print("3 deep layers, 128 kernels per layer, 0.6 dropout rate")
train_scores_reg3_1 = reg_model3_1.evaluate(X3_1_train_scaled, y3_1_train_final, verbose=0)
val_scores_reg3_1   = reg_model3_1.evaluate(X3_1_val_scaled, y3_1_val, verbose=0)
print(f"Train MAE: {train_scores_reg3_1[1]:.4f}, Train MSE: {train_scores_reg3_1[2]:.4f}")
print(f"Val   MAE: {val_scores_reg3_1[1]:.4f}, Val   MSE: {val_scores_reg3_1[2]:.4f}")


The original regularized model continues to have the best MAE.

Max Rank Change

In [ ]:
# adding a layer 

l2_reg = 1e-4
dropout_rate = 0.4

reg_model4_1 = keras.Sequential([
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg),
                 input_shape=(X4_1_train.shape[1],)),
    layers.Dropout(dropout_rate),
        
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(1)  # Single output for regression
])

reg_model4_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_reg_model4_1 = reg_model4_1.fit(
    X4_1_train_scaled, y4_1_train_final,
    validation_data=(X4_1_val_scaled, y4_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
print("=== Regularized Model (L2 + Dropout) ===")
print("    Adding a layer")
train_scores_reg4_1 = reg_model4_1.evaluate(X4_1_train_scaled, y4_1_train_final, verbose=0)
val_scores_reg4_1   = reg_model4_1.evaluate(X4_1_val_scaled, y4_1_val, verbose=0)
print(f"Train MAE: {train_scores_reg4_1[1]:.4f}, Train MSE: {train_scores_reg4_1[2]:.4f}")
print(f"Val   MAE: {val_scores_reg4_1[1]:.4f}, Val   MSE: {val_scores_reg4_1[2]:.4f}")


These results are very nearly identical to the initial configurations. Doubling the kernels in each layer.

In [ ]:
# adding a layer 

l2_reg = 1e-4
dropout_rate = 0.4

reg_model4_1 = keras.Sequential([
    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg),
                 input_shape=(X4_1_train.shape[1],)),
    layers.Dropout(dropout_rate),
        
    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(1)  # Single output for regression
])

reg_model4_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_reg_model4_1 = reg_model4_1.fit(
    X4_1_train_scaled, y4_1_train_final,
    validation_data=(X4_1_val_scaled, y4_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
print("=== Regularized Model (L2 + Dropout) ===")
print("    Adding a layer")
train_scores_reg4_1 = reg_model4_1.evaluate(X4_1_train_scaled, y4_1_train_final, verbose=0)
val_scores_reg4_1   = reg_model4_1.evaluate(X4_1_val_scaled, y4_1_val, verbose=0)
print(f"Train MAE: {train_scores_reg4_1[1]:.4f}, Train MSE: {train_scores_reg4_1[2]:.4f}")
print(f"Val   MAE: {val_scores_reg4_1[1]:.4f}, Val   MSE: {val_scores_reg4_1[2]:.4f}")

The original regularized model has the best MAE scores.

## Evaluation

### Summary of Model Performance

**XGBoost**
The XGBoost model did not return meaningful results for this dataset. The best metrics for XGBoost were:

_Maximum Peak Position_

- RMSE: 22.35
- r<sup>2</sup>: 0.138

_Maximum Rank Increase_

- RMSE: 11.73
- - r<sup>2</sup>: 0.002

**k-Nearest Neighbors**
This model also did not perform well on this data set. Its best metrics were:

_Maximum Peak Position_

- Accuracy: 0.052

_Maximum Rank Increase_

- Accuracy: 0.217

**Deep Learning**

The deep learning model returned the most promising results and are described in the next section.

### Final Models

The final models are both based on a deep learning architecture. 

_Maximum Peak Position_

The best model for the Maximum Peak Position is named reg_model3_1. It is a 3-layer, regularized model trained on the song dataset that does not contain genre information.Its final metrics were:

- Training mean absolute error: 16.99
- Validation mean absolute error: 17.67

_Maximum Rank Increase_

The best model for the Maximum Rank Increase is named reg_model4_2. It is also a 3-layer, regularized model trained on the song dataset without genre.
Its final metrics:

- Training mean absolute error: 7.75
- Validation mean absolute error: 8.91

### Business Insight/Recommendation 1

The genre of a song, at least how it is captured on Spotify, has very little influence on its movement on the Billboard Hot 100 list. This gives FutureProduct Advisors consultants and their customers significant leeway when they are selecting music for their social and advertising campaigns.

### Business Insight/Recommendation 2

The characteristics of a given song appear to be the most important factors in its performance on the Hot 100 list. Features such as tempo, danceability, etc. have the most predictive power for the Hot 100 list.

### Business Insight/Recommendation 3

Digital music has a large number of characteristics that can be pulled into machine learning models. The factors that have the largest influence on popularity are often unexpected, and modeling and analysis of songs requires methodical and careful selection and vetting of features.

## Conclusion and Next Steps

reg_model3_1 (predicting highest ranking on the Billboard Hot 100) and reg_model4_2 (predicting largest week over week ranking increase) are ready for initial deployment to FutureProduct Advisors. 

Detailed training will be provided, but at a high level here is how users will engage:

1. Consultants will identify a list of songs they'd like to explore.
    - We recommend using the songs in places 90-100 of the Billboard Hot 100 at a minimum
2. Consultants download those songs' metadata from Spotify as a CSV file (detailed instructions to come)
3. Consultants load the CSV file into these models, and the models will return predicted activity for each song.

There is also an additional opportunity to built on and refine these models by indluding additional data and experimenting with other modeling approaches. If the FutureProduct Advisors consultants have positive feedback on these prototype tools, we will be happy to partner on future enhancements.
